# Give your code a structure

At some point in your python programming experience, you will start having continuous *deja-vu*'s, as you will find yourself coding the same algorithm over and over in different scripts. When people get to that point, they typically resort to copy-pasting the algorithm from some previous code, and then perhaps modifying it to adapt it to the needs of the new code. This often happens even within a group of script that belong to the same project, but this is **bad practice** which you should start avoiding as soon as possible, because it inevitably leads to the following situation: one day, you realize that the algorithm contained a bug, or simply it could be made more efficient, and then you have to change the code in *all the places you've copied it all around your scripts*, with a very high probability that you will end up having different versions of it co-living in your code. A disaster!

In this brief lesson, we will see how one can avoid this bad practice by structuring the code as a *python package* and ultimately by importing your packages whenever you need them. The next step will be that of using a *version control system* (VCS) such as `git` to keep track of the evolution of your package and to be sure that you always know which version of your code you are using.

## Python package folder structure

Starting with `python 3`, the folder structure where your scripts live in has become the main way to tell python that these constitute a package altogether. The idea is essentially that a folder represents a class, and this is made explicit by adding an `__init__.py` script into that folder, which behaves very much like the `def __init__()` method of a class: it serves to initialize the module when you import it.

As an example, say that we want to create a python module that computes *credible intervals* from parameter samples (a very typical situation nowadays!). You want to be able to construct both 1D credible intervals and 2D credible contours by binning your samples (obtained e.g. by a Markov Chain Monte Carlo) and applying a `greedy binning` algorithm, that is, you want to sum the bin frequencies in descending order until you reach a given fraction of the total samples (which represents your credible level). You may define a function that does so with a 1D array of samples, something like this:

In [5]:
import numpy as np

def greedy_credible_1D(x,bins=50,level=0.68):
    """
    Compute a 1D credible interval using a greedy binning algorithm.
    
    Parameters:
    - x: 1D array of parameter samples
    - bins: array of bin edges, or number of equally spaced bins
    - level: the desired credible level
    
    Returns:
    - c_edges: the edges of the credible interval (an even number)
    """
    # create histogram f x
    h,edges = np.histogram(x,bins=bins)
    
    # create sorting index
    s = np.argsort(h)
    # create cumulative sum (normalized)
    csum = np.zeros_like(h)
    csum[s]=np.cumsum(h[s])
    csum=csum/csum.max()
    
    # compute credible range edges
    return edges,csum>=(1.-level)

You may want to use this piece of code (and others regarding the same argument) in a diverse range of programs: you therefore better construct a python package to hold it. This is essentially done by placing a python script containing this code into a folder (let's call the folder `credible` and the script `greedy_algorithms.py`), and then adding a `__init__.py` script along it. The latter script will have the role of defining what is exposed to the user, and what is not, by means of a series of imports:

```
# file greedy_algorithms.py

def greedy_credible_1D(x,bins=50,level=0.68):
    """
    Compute a 1D credible interval using a greedy binning algorithm.
    
    Parameters:
    - x: 1D array of parameter samples
    - bins: array of bin edges, or number of equally spaced bins
    - level: the desired credible level`
    
    Returns:
    - c_edges: the edges of the credible interval (an even number)
    """
    # create histogram f x
    h,edges = np.histogram(x,bins=bins)
    
    # create sorting index
    s = np.argsort(h)
    # create cumulative sum (normalized)
    csum = np.zeros_like(h)
    csum[s]=np.cumsum(h[s])
    csum=csum/csum.max()
    
    # compute credible range edges
    return edges,csum>=(1.-level)`
```

```
# file __init__.py

from .greedy_algorithms import greedy_credible_1D
```

You may then want to add another script that holds definitions for standard credible levels such as "one sigma", "two sigma" and "three sigma":

```
# file sigmas.py

one_sigma = 0.68
two_sigma = 0.95
three_sigma = 0.9973
```

You can then import this into the init script to make them available to the user:

```
# file __init__.py

from .greedy_algorithms import greedy_credible_1D
from .sigmas import one_sigma,two_sigma,three_sigma
```